[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Hawksight-AI/semantica/blob/main/cookbook/advanced/09_Semantic_Layer_Construction.ipynb)

# Semantic Layer Construction

## Overview

Build an enterprise semantic layer: construct knowledge graph, generate ontology, create semantic layer, export RDF, and store in triplet store.


**Documentation**: [API Reference](https://semantica.readthedocs.io/concepts/)

## Installation

Install Semantica from PyPI:

```bash
pip install semantica
# Or with all optional dependencies:
pip install semantica[all]
```

## Workflow: Build KG → Generate Ontology → Create Semantic Layer → Export RDF 


In [ ]:
!pip install -qU semantica


In [ ]:
from semantica.kg import GraphBuilder
from semantica.ontology import OntologyGenerator
from semantica.export import RDFExporter
from semantica.triplet_store import TripletStore


## Step 1: Build Knowledge Graph


In [ ]:
builder = GraphBuilder()

entities = [
    {"id": "e1", "type": "Person", "name": "Alice", "properties": {"age": 30, "role": "Engineer"}},
    {"id": "e2", "type": "Person", "name": "Bob", "properties": {"age": 35, "role": "Manager"}},
    {"id": "e3", "type": "Organization", "name": "Tech Corp", "properties": {"founded": 2010}},
    {"id": "e4", "type": "Project", "name": "Project Alpha", "properties": {"status": "active"}},
]

relationships = [
    {"source": "e1", "target": "e2", "type": "reports_to"},
    {"source": "e1", "target": "e3", "type": "works_for"},
    {"source": "e2", "target": "e3", "type": "works_for"},
    {"source": "e1", "target": "e4", "type": "works_on"},
]

knowledge_graph = builder.build(entities, relationships)


## Step 2: Generate Ontology


In [ ]:
generator = OntologyGenerator()
ontology = generator.generate_from_graph(knowledge_graph)


## Step 3: Create Semantic Layer


In [ ]:
def create_mappings(kg, ontology):
    mappings = {
        "entity_type_mappings": {},
        "relationship_type_mappings": {},
        "property_mappings": {}
    }
    
    entity_types = set(e.get("type") for e in entities)
    ontology_classes = ontology.get("classes", [])
    
    for entity_type in entity_types:
        matching_class = next((cls for cls in ontology_classes if cls.get("name") == entity_type), None)
        if matching_class:
            mappings["entity_type_mappings"][entity_type] = matching_class.get("uri", entity_type)
    
    relationship_types = set(r.get("type") for r in relationships)
    ontology_properties = ontology.get("properties", [])
    
    for rel_type in relationship_types:
        matching_prop = next((prop for prop in ontology_properties if prop.get("name") == rel_type), None)
        if matching_prop:
            mappings["relationship_type_mappings"][rel_type] = matching_prop.get("uri", rel_type)
    
    return mappings

mappings = create_mappings(knowledge_graph, ontology)

semantic_layer = {
    "graph": knowledge_graph,
    "ontology": ontology,
    "mappings": mappings,
    "metadata": {
        "version": "1.0",
        "created_at": "2024-01-01",
        "description": "Enterprise semantic layer"
    }
}


## Step 4: Export RDF


In [ ]:
exporter = RDFExporter()
# Export Knowledge Graph
exporter.export(knowledge_graph, "knowledge_graph.ttl", format="turtle")
print("Exported knowledge graph to knowledge_graph.ttl")


## Summary

Enterprise semantic layer construction:
- Knowledge Graph Built
- Ontology Generated
- Semantic Layer Created with Mappings
- RDF Export Completed
